# Proyecto Final de Organización y Arquitectura de Computadora
## Interrupciones y Control de Procesos en el Desarrollo y Ejecución de Programa
### Ferrer, Sebastián. CED
### Lucero, Pedro. 8-973-1067

### Plan:
De lo que voy pensando podemos tener
- Ejecución - Clase
- anhadirInterrupcion
- verDetalles (Tipo las puras interrupciones con cada T)
- verResultado (Que es la tabla desarrollada)
- verBitacora
- (Very loco y solo si hay tiempo) meterle pandas pa csv

### Prueba pre-armada:
- Duracion de programa = 60
- N interrupciones con (T, duración de) = (), (), (), ()
- T final debe ser de: [idklol]

In [1]:
class Interrupcion:
    IRQ = 99
    prioridad = 99
    funcion = "Default"
    def __init__(self, IRQ, prioridad, funcion):
        self.IRQ = IRQ
        self.prioridad = prioridad
        self.funcion = funcion
    def __repr__(self):
        return f"I({self.funcion})"

In [2]:
def_irqs = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
def_prios = [1, 2, 11, 12, 13, 14, 15, 3, 4, 5, 6, 7, 8, 9, 10]
def_funcs = ["Reloj del sistema", 
             "Teclado",
             "COM2 y COM4",
             "COM1 y COM3",
             "Libre(5)",
             "Controlador Floppy - Diskette",
             "Puerto Paralelo - Impresora",
             "Reloj (tics) en tiempo real CMOS",
             "Red, sonido, puerto SCSI",
             "Libre(10)",
             "Libre(11)",
             "PS-mouse",
             "Co-procesador matemático",
             "Canal IDE primario(Disco)",
             "Libre(15)"]

interrupciones = {irq: Interrupcion(irq, prio, func)
                  for irq, prio, func in zip(def_irqs, def_prios, def_funcs)}

In [37]:
from collections import deque # lo usamos para implementar la pila de interrupciones
class Ejecucion:
    def __init__(self, duracion_programa):
        self.duracion_programa = duracion_programa
        self.interrupciones = [] # este es un arreglo de tuplas para manejar T's con duraciones
        # print(self.duracion_programa)
    
    def anhadir_interrupcion(self, timestamp, irq, duracion): # Añadimos las interrupciones y ordenamos
        # Interrupciones: (Timestamp redondeado a 2 decimales | IRQ de la int. | duración (en seg.) de la int.)
        self.interrupciones.append((round(timestamp, 2), interrupciones[irq], duracion))
        self.interrupciones.sort(key=lambda int: int[0])

    def calcular(self):
        cola = deque() # cola: (prioridad, t_restante)
        cola.append([999, self.duracion_programa])
        
        T = 0
        int_index = 0
        while len(cola) > 0 and int_index < len(self.interrupciones):
            print(T)
            actual = cola.pop()
            sig_T = self.interrupciones[int_index][0]
            sig_prio = self.interrupciones[int_index][1].prioridad
            sig_t_res = self.interrupciones[int_index][2]
            siguiente = [sig_prio, sig_t_res]
            int_index += 1
            
            delta_T = sig_T - T
            
            if actual[1] - delta_T <= 0:
                T += actual[1]
                continue
            
            actual[1] -= sig_T - T
            T += sig_T
            
            # 1. Prio sig < actual → sig. "gana"
            if siguiente[0] < actual[0]:
                cola.append(actual)
                cola.append(siguiente)
            
            # 2. Prio sig > actual → actual "gana"
            if siguiente[0] > actual[0]:
                cola.append(siguiente)
                cola.append(actual)
            
            
        print(cola)

In [38]:
# Prueba pre armada, con lo definido antes
prueba1 = Ejecucion(15)
prueba1.anhadir_interrupcion(3, 14, 5)
prueba1.anhadir_interrupcion(5, 9, 10)
prueba1.anhadir_interrupcion(10, 7, 8)
prueba1.anhadir_interrupcion(24, 14, 12)
prueba1.anhadir_interrupcion(42, 9, 5)
prueba1.interrupciones

[(3, I(Canal IDE primario(Disco)), 5),
 (5, I(Red, sonido, puerto SCSI), 10),
 (10, I(Puerto Paralelo - Impresora), 8),
 (24, I(Canal IDE primario(Disco)), 12),
 (42, I(Red, sonido, puerto SCSI), 5)]

In [39]:
prueba1.calcular()

0
3
8
18
42
deque([[999, 12], [9, 3], [15, 8], [9, 12]])


In [36]:
a = deque()
a.append(1)
a.append(2)
a.append(3)

In [9]:
b = a.pop()
b

3

In [ ]:
        cola = deque() # cola: (prioridad, t_restante)
        # Iniciamos ↓ en 0
        recorrido = 0 + self.interrupciones[0][0] # El programa recorre hasta la primera int.
        
        T = recorrido
        for orden, inte in enumerate(self.interrupciones):
            siguiente = self.interrupciones[orden+1]
            if len(cola) == 0:
                t_restante = T + inte[2] - siguiente[0]
                if t_restante > 0:
                    cola.append((inte[1].prioridad, t_restante))
            else:
                if siguiente[1].prioridad > inte[1].prioridad:
                    # cola.append((inte[1].prioridad, t_restante))
                else: